# Laminet Prototype

This notebook implements a minimal working prototype for a Laminet model - a neural architecture that models sequential information as a continuous semantic field rather than discrete tokens.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import matplotlib.animation as animation
from IPython.display import HTML

## 1. Defining the Semantic Field

We'll first implement the core field components: field points and the semantic field itself.

In [ ]:
class FieldPoint(nn.Module):
    def __init__(self, embed_dim):
        super().__init__()
        # Position vector in semantic space
        self.position = nn.Parameter(torch.randn(embed_dim))
        # Velocity vector for field evolution
        self.velocity = nn.Parameter(torch.zeros(embed_dim))
        # Mass affects how much the point is influenced by forces
        self.mass = nn.Parameter(torch.ones(1))
        # Semantic charge determines attraction/repulsion properties
        self.charge = nn.Parameter(torch.randn(1))
        # Decay rate for velocity (damping)
        self.decay_rate = nn.Parameter(torch.tensor([0.1]))

In [ ]:
class SemanticField(nn.Module):
    def __init__(self, num_points, embed_dim):
        super().__init__()
        # Create a collection of field points
        self.points = nn.ModuleList([FieldPoint(embed_dim) for _ in range(num_points)])
        self.embed_dim = embed_dim
        # Learnable parameters for force scaling
        self.attraction_scale = nn.Parameter(torch.tensor([1.0]))
        self.repulsion_scale = nn.Parameter(torch.tensor([0.5]))
        
        # Store field evolution history for visualization
        self.evolution_history = []
    
    def embed_from_input(self, embeddings):
        """Initialize field points from input embeddings"""
        for i, point in enumerate(self.points):
            if i < len(embeddings):
                point.position.data = embeddings[i]
        # Reset evolution history
        self.evolution_history = [self.get_current_state()]
    
    def get_current_state(self):
        """Get current positions and charges of all points"""
        positions = torch.stack([p.position.detach() for p in self.points])
        charges = torch.stack([p.charge.detach() for p in self.points])
        return {
            'positions': positions,
            'charges': charges
        }
    
    def semantic_attraction(self, point_i, point_j):
        """Calculate semantic attraction force between two field points"""
        direction = point_j.position - point_i.position
        distance = direction.norm(p=2) + 1e-6  # Avoid division by zero
        
        # Attraction force based on semantic similarity
        force_magnitude = (point_i.charge * point_j.charge) / (distance**2)
        attraction = self.attraction_scale * force_magnitude * direction / distance
        
        # Add repulsion for very close points (prevent collapse)
        repulsion_magnitude = self.repulsion_scale / (distance**3 + 1e-6)
        repulsion = -repulsion_magnitude * direction / distance
        
        return attraction + repulsion
    
    def compute_net_force(self, point_idx):
        """Calculate net force for a specific field point"""
        forces = []
        for j, other_point in enumerate(self.points):
            if j == point_idx:
                continue
            forces.append(self.semantic_attraction(self.points[point_idx], other_point))
        
        if forces:
            return torch.sum(torch.stack(forces), dim=0)
        return torch.zeros_like(self.points[point_idx].position)
    
    def evolve(self, dt=0.01):
        """Evolve field points based on forces for a single timestep"""
        for idx, point in enumerate(self.points):
            # Calculate net force
            net_force = self.compute_net_force(idx)
            
            # Update velocity (F = ma -> a = F/m)
            point.velocity = point.velocity + (net_force / point.mass.abs().clamp(min=0.1)) * dt
            
            # Update position
            point.position = point.position + point.velocity * dt
            
            # Apply velocity decay (damping)
            point.velocity *= (1.0 - point.decay_rate.abs().clamp(max=0.2) * dt)
        
        # Store current state in history
        self.evolution_history.append(self.get_current_state())
    
    def measure_potential_energy(self):
        """Calculate potential energy of the field (kinetic energy of points)"""
        energy = 0.0
        for point in self.points:
            # Kinetic energy: 0.5 * m * v^2
            energy += 0.5 * point.mass.abs() * (point.velocity.norm(p=2) ** 2)
        return energy
    
    def get_field_encoding(self):
        """Get field encoding as mean of all point positions"""
        positions = torch.stack([p.position for p in self.points])
        return positions.mean(dim=0)

## 2. Implementation of the Laminet Model

Now we'll implement the full Laminet model which wraps around the semantic field and provides end-to-end functionality.

In [ ]:
class Laminet(nn.Module):
    def __init__(self, input_dim=32, embed_dim=64, output_dim=32, num_field_points=5):
        super().__init__()
        
        # Define model components
        self.input_dim = input_dim
        self.embed_dim = embed_dim
        self.output_dim = output_dim
        
        # Field initialization module (maps input to initial field)
        self.embedder = nn.Sequential(
            nn.Linear(input_dim, embed_dim * 2),
            nn.LayerNorm(embed_dim * 2),
            nn.LeakyReLU(),
            nn.Linear(embed_dim * 2, embed_dim)
        )
        
        # Field evolution module
        self.field = SemanticField(num_points=num_field_points, embed_dim=embed_dim)
        
        # Field decoding module (maps evolved field to output)
        self.decoder = nn.Sequential(
            nn.Linear(embed_dim, embed_dim * 2),
            nn.LayerNorm(embed_dim * 2),
            nn.LeakyReLU(),
            nn.Linear(embed_dim * 2, output_dim)
        )
    
    def forward(self, x, time_steps=20, dt=0.01):
        """Forward pass through Laminet model"""
        batch_size = x.shape[0]
        
        # Process inputs in batch
        outputs = []
        energies = []
        
        for i in range(batch_size):
            # Initialize field from input
            input_sample = x[i]
            
            # Reshape input to initialize multiple field points if needed
            if len(input_sample.shape) == 1 and self.field.points:
                # Split vector into chunks for each field point
                chunk_size = input_sample.shape[0] // len(self.field.points)
                if chunk_size > 0:
                    chunks = input_sample.split(chunk_size)
                    embeddings = [self.embedder(chunk) for chunk in chunks[:len(self.field.points)]]
                else:
                    # If input is too small to split, duplicate it
                    embeddings = [self.embedder(input_sample) for _ in range(len(self.field.points))]
            else:
                # Single embedding for single field point
                embeddings = [self.embedder(input_sample)]
            
            # Initialize field with embeddings
            self.field.embed_from_input(embeddings)
            
            # Evolve field over time
            for _ in range(time_steps):
                self.field.evolve(dt)
            
            # Get field encoding and decode output
            field_encoding = self.field.get_field_encoding()
            output = self.decoder(field_encoding)
            
            # Store results
            outputs.append(output)
            energies.append(self.field.measure_potential_energy())
        
        # Stack results into batch
        outputs = torch.stack(outputs)
        energies = torch.stack(energies)
        
        return outputs, energies

## 3. Generating a Synthetic Dataset

Let's create a simple synthetic dataset for our semantic evolution task (e.g., evolving a concept like "cold" to "warmth").

In [ ]:
def generate_synthetic_dataset(num_samples=1000, input_dim=32, output_dim=32, num_concepts=3):
    """Generate a synthetic dataset for concept evolution"""
    # Create synthetic concept vectors
    np.random.seed(42)
    concepts = {}
    
    # Generate random concept names for our toy dataset
    concept_names = [
        "cold", "cool", "neutral", "warm", "hot",
        "sad", "melancholy", "neutral", "content", "happy",
        "simple", "basic", "moderate", "complex", "intricate"
    ]
    
    # Create random concept embeddings
    for name in concept_names:
        vec = np.random.randn(input_dim)
        vec = vec / np.linalg.norm(vec)  # Normalize
        concepts[name] = vec
    
    # Create concept sequences (paths through concept space)
    sequences = [
        ["cold", "cool", "neutral", "warm", "hot"],
        ["sad", "melancholy", "neutral", "content", "happy"],
        ["simple", "basic", "moderate", "complex", "intricate"]
    ]
    
    # Generate samples
    X = []
    y = []
    
    for _ in range(num_samples):
        # Choose a random sequence
        seq_idx = np.random.randint(0, len(sequences))
        sequence = sequences[seq_idx]
        
        # Choose a random starting position in the sequence
        start_idx = np.random.randint(0, len(sequence) - 1)
        
        # Get input and target concepts
        input_concept = sequence[start_idx]
        target_concept = sequence[start_idx + 1]
        
        # Add some noise to make the task more challenging
        input_vec = concepts[input_concept] + np.random.randn(input_dim) * 0.1
        target_vec = concepts[target_concept] + np.random.randn(output_dim) * 0.1
        
        X.append(input_vec)
        y.append(target_vec)
    
    # Convert to torch tensors
    X = torch.tensor(np.array(X), dtype=torch.float32)
    y = torch.tensor(np.array(y), dtype=torch.float32)
    
    # Create a concept dictionary for evaluation
    concept_tensors = {name: torch.tensor(vec, dtype=torch.float32) for name, vec in concepts.items()}
    
    return X, y, concept_tensors, sequences

## 4. Training Loop

In [ ]:
def train_laminet(model, X_train, y_train, num_epochs=100, batch_size=16, lr=0.001):
    """Train Laminet model"""
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    # Track losses
    losses = []
    energy_values = []
    
    # Create data loader
    dataset = torch.utils.data.TensorDataset(X_train, y_train)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    for epoch in range(num_epochs):
        epoch_loss = 0
        epoch_energy = 0
        
        for batch_X, batch_y in dataloader:
            # Forward pass
            optimizer.zero_grad()
            outputs, energy = model(batch_X)
            
            # Calculate loss
            loss = criterion(outputs, batch_y)
            
            # Add regularization based on field energy (optional)
            energy_loss = 0.01 * energy.mean()
            total_loss = loss + energy_loss
            
            # Backward pass and optimize
            total_loss.backward()
            optimizer.step()
            
            # Track metrics
            epoch_loss += loss.item()
            epoch_energy += energy.mean().item()
        
        # Record epoch metrics
        avg_loss = epoch_loss / len(dataloader)
        avg_energy = epoch_energy / len(dataloader)
        losses.append(avg_loss)
        energy_values.append(avg_energy)
        
        if (epoch+1) % 10 == 0:
            print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.6f}, Energy: {avg_energy:.6f}")
    
    return losses, energy_values

## 5. Visualization Functions

In [ ]:
def plot_training_curves(losses, energies):
    """Plot training metrics"""
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.plot(losses)
    plt.title('Training Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    
    plt.subplot(1, 2, 2)
    plt.plot(energies)
    plt.title('Field Energy')
    plt.xlabel('Epoch')
    plt.ylabel('Energy')
    
    plt.tight_layout()
    plt.show()

def visualize_field_evolution(field_history):
    """Create PCA visualization of field evolution"""
    # Extract positions from history
    all_positions = torch.cat([state['positions'] for state in field_history])
    
    # Apply PCA to reduce to 2D for visualization
    pca = PCA(n_components=2)
    positions_np = all_positions.numpy()
    pca_result = pca.fit_transform(positions_np)
    
    # Reshape back to timesteps and points
    num_timesteps = len(field_history)
    num_points = field_history[0]['positions'].shape[0]
    pca_result = pca_result.reshape(num_timesteps, num_points, 2)
    
    # Extract charges
    charges = [state['charges'].numpy() for state in field_history]
    
    # Create animation
    fig, ax = plt.subplots(figsize=(8, 8))
    
    def update(frame):
        ax.clear()
        # Plot field points
        scatter = ax.scatter(pca_result[frame, :, 0], pca_result[frame, :, 1], 
                  c=charges[frame].flatten(), cmap='coolwarm', 
                  s=100, alpha=0.7, vmin=-1, vmax=1)
        
        # Add trajectories (last 5 steps)
        start_idx = max(0, frame - 5)
        for i in range(num_points):
            traj_x = pca_result[start_idx:frame+1, i, 0]
            traj_y = pca_result[start_idx:frame+1, i, 1]
            ax.plot(traj_x, traj_y, 'k-', alpha=0.3)
        
        # Add some styling
        ax.set_xlim(pca_result[:, :, 0].min() - 0.5, pca_result[:, :, 0].max() + 0.5)
        ax.set_ylim(pca_result[:, :, 1].min() - 0.5, pca_result[:, :, 1].max() + 0.5)
        ax.set_title(f'Field Evolution - Step {frame+1}/{num_timesteps}')
        ax.grid(alpha=0.3)
        
        return scatter,
    
    ani = animation.FuncAnimation(fig, update, frames=num_timesteps, interval=100, blit=True)
    plt.close()  # Prevent double display in Jupyter
    return HTML(ani.to_jshtml())

## 6. Evaluation Functions

In [ ]:
def find_nearest_concept(embedding, concept_dict):
    """Find the nearest concept to a given embedding"""
    min_dist = float('inf')
    nearest = None
    
    for name, vec in concept_dict.items():
        dist = torch.norm(embedding - vec, p=2)
        if dist < min_dist:
            min_dist = dist
            nearest = name
    
    return nearest, min_dist.item()

def evaluate_concept_transitions(model, concept_dict, sequences):
    """Evaluate how well the model transitions between concepts"""
    results = []
    
    for sequence in sequences:
        seq_results = []
        
        for i in range(len(sequence) - 1):
            start_concept = sequence[i]
            expected_next = sequence[i + 1]
            
            # Get embedding for start concept
            input_embed = concept_dict[start_concept].unsqueeze(0)
            
            # Evolve through model
            with torch.no_grad():
                output, _ = model(input_embed)
            
            # Find nearest concept
            predicted_concept, distance = find_nearest_concept(output[0], concept_dict)
            
            seq_results.append({
                'input': start_concept,
                'expected': expected_next,
                'predicted': predicted_concept,
                'distance': distance
            })
        
        results.append(seq_results)
    
    return results

## 7. Full Training and Evaluation

In [ ]:
# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Generate synthetic dataset
input_dim = 32
output_dim = 32
embed_dim = 64
num_field_points = 5

X, y, concept_dict, sequences = generate_synthetic_dataset(
    num_samples=1000, 
    input_dim=input_dim, 
    output_dim=output_dim
)

# Split train/test
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

print(f"Dataset created with {len(X_train)} training and {len(X_test)} test samples")

# Create Laminet model
model = Laminet(
    input_dim=input_dim,
    embed_dim=embed_dim,
    output_dim=output_dim,
    num_field_points=num_field_points
)

# Print model parameter count
param_count = sum(p.numel() for p in model.parameters())
print(f"Model created with {param_count} parameters")

In [ ]:
# Train the model
losses, energies = train_laminet(
    model=model,
    X_train=X_train,
    y_train=y_train,
    num_epochs=100,
    batch_size=16,
    lr=0.001
)

# Plot training curves
plot_training_curves(losses, energies)

## 8. Visualizing Field Evolution

In [ ]:
# Choose an example input to visualize field evolution
sample_idx = 0
sample_input = X_test[sample_idx].unsqueeze(0)

# Forward pass to get field evolution
with torch.no_grad():
    outputs, _ = model(sample_input)

# Visualize the field evolution
animation = visualize_field_evolution(model.field.evolution_history)
animation

## 9. Evaluating Concept Transitions

In [ ]:
# Evaluate concept transitions
transition_results = evaluate_concept_transitions(model, concept_dict, sequences)

# Display results
for i, sequence_results in enumerate(transition_results):
    print(f"Sequence {i+1}: {' -> '.join(sequences[i])}")
    correct = 0
    
    for result in sequence_results:
        match = result['expected'] == result['predicted']
        correct += int(match)
        print(f"  {result['input']} -> {result['predicted']} (expected: {result['expected']}) {'✓' if match else '✗'}")
    
    accuracy = correct / len(sequence_results) * 100
    print(f"  Accuracy: {accuracy:.2f}%\n")

## 10. Testing End-to-End Concept Evolution

In [ ]:
def evolve_concept_chain(model, start_concept, num_steps=5):
    """Evolve a concept through multiple steps"""
    current_embedding = concept_dict[start_concept].unsqueeze(0)
    evolution = [start_concept]
    
    for step in range(num_steps):
        # Evolve through model
        with torch.no_grad():
            output, _ = model(current_embedding)
        
        # Find nearest concept
        predicted_concept, _ = find_nearest_concept(output[0], concept_dict)
        evolution.append(predicted_concept)
        
        # Use output as next input
        current_embedding = output
    
    return evolution

# Test chaining multiple evolutions
for sequence in sequences:
    start_concept = sequence[0]
    evolution = evolve_concept_chain(model, start_concept, num_steps=5)
    print(f"Starting with '{start_concept}' evolved to: {' -> '.join(evolution)}")

## 11. Conclusion

We've successfully implemented a minimal working Laminet model that:

1. Represents sequential information as a continuous semantic field
2. Evolves the field using simulated forces and dynamics
3. Can transform inputs through semantic space along meaningful trajectories
4. Visualizes the evolution of semantic fields over time

This prototype demonstrates the core concepts behind field evolution models as an alternative to discrete token-based approaches. The next steps would be to:

1. Scale up to handle more complex inputs
2. Implement more sophisticated field dynamics
3. Test on real-world language understanding tasks
4. Compare performance with traditional transformer models